# Проект по SQL

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Описание-проекта" data-toc-modified-id="Описание-проекта-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Описание проекта</a></span></li><li><span><a href="#Описание-данных" data-toc-modified-id="Описание-данных-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Описание данных</a></span></li><li><span><a href="#Подключение-к-базе-данных" data-toc-modified-id="Подключение-к-базе-данных-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Подключение к базе данных</a></span></li><li><span><a href="#Запрос-№1" data-toc-modified-id="Запрос-№1-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Запрос №1</a></span></li><li><span><a href="#Запрос-№2" data-toc-modified-id="Запрос-№2-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Запрос №2</a></span></li><li><span><a href="#Запрос-№3" data-toc-modified-id="Запрос-№3-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Запрос №3</a></span></li><li><span><a href="#Запрос-№4" data-toc-modified-id="Запрос-№4-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Запрос №4</a></span></li><li><span><a href="#Запрос-№5" data-toc-modified-id="Запрос-№5-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Запрос №5</a></span></li><li><span><a href="#Итоговый-вывод" data-toc-modified-id="Итоговый-вывод-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Итоговый вывод</a></span></li></ul></div>

## Описание проекта

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.

Компания-заказчик решила быть на волне и купила крупный сервис для чтения книг по подписке.


**Задача** — написать запросы на SQL для того, чтобы получить данные, которые помогут сформулировать ценностное предложение для нового продукта.


**Исходные данные** — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг.

## Описание данных

**Таблица `books`**

Содержит данные о книгах.

**Таблица `authors`**

Содержит данные об авторах.

**Таблица `publishers`**

Содержит данные об издательствах.

**Таблица `ratings`**

Содержит данные о пользовательских оценках книг.

**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги.

## Подключение к базе данных

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

In [2]:
# устанавливаем параметры
db_config = {'user': '', # имя пользователя
'pwd': '', # пароль
'host': '',
'port': , # порт подключения
'db': ''} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

## Запрос №1

- Посчитайте, сколько книг вышло после 1 января 2000 года;

In [3]:
# чтобы выполнить SQL-запрос, используем Pandas
query = '''
SELECT COUNT(book_id)
FROM books
WHERE publication_date > '2000-01-01';
'''

pd.io.sql.read_sql(sql=text(query), con = engine)

,count
0,819


*Вывод:* После 1 января 2000 года вышло 819 книг.

## Запрос №2

- Для каждой книги посчитайте количество обзоров и среднюю оценку;

In [4]:
query = ''' 
SELECT b.title AS book_name,
       b.book_id AS id,
       sub_rev.n_rev AS reviews,
       sub_rat.avg_rat AS avg_rating
FROM books b LEFT JOIN (SELECT book_id,
                               AVG(rating) AS avg_rat
                        FROM ratings
                        GROUP BY book_id) AS sub_rat ON sub_rat.book_id = b.book_id
             LEFT JOIN (SELECT book_id,
                               COUNT(review_id) AS n_rev
                        FROM reviews
                        GROUP BY book_id) AS sub_rev ON sub_rev.book_id = b.book_id
ORDER BY 2;
''' 

pd.io.sql.read_sql(sql=text(query), con = engine)

,book_name,id,reviews,avg_rating
0,'Salem's Lot,1,2.0,3.666667
1,1 000 Places to See Before You Die,2,1.0,2.500000
2,13 Little Blue Envelopes (Little Blue Envelope...,3,3.0,4.666667
3,1491: New Revelations of the Americas Before C...,4,2.0,4.500000
4,1776,5,4.0,4.000000
...,...,...,...,...
995,Wyrd Sisters (Discworld #6; Witches #2),996,3.0,3.666667
996,Xenocide (Ender's Saga #3),997,3.0,3.400000
997,Year of Wonders,998,4.0,3.200000
998,You Suck (A Love Story #2),999,2.0,4.500000


*Вывод:* для решения задачи была сформирована таблица, содержащая информацию об идентификаторах всех книг, их названиях, количестве обзоров и среднем значении рейтинга. Данные отсортированы по идентификаторам книг.

## Запрос №3

- Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры.

In [5]:
query = '''
SELECT b.publisher_id,
       publisher,
       COUNT(book_id) books_count
FROM books b
LEFT OUTER JOIN publishers p ON b.publisher_id = p.publisher_id
WHERE num_pages > 50
GROUP BY 1, 2
ORDER BY 3 DESC
LIMIT 1;
'''

pd.io.sql.read_sql(sql=text(query), con = engine)

,publisher_id,publisher,books_count
0,212,Penguin Books,42


*Вывод:* издательство, которое выпустило наибольшее число книг толще 50 страниц — Penguin Books (42 книги).

## Запрос №4

- Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками.

In [6]:
query = '''
WITH pt AS (SELECT a.author_id,
                   author,
                   b.book_id,
                   title,
                   AVG(rating) avg_rating
            FROM books b
            LEFT OUTER JOIN ratings r ON b.book_id = r.book_id
            LEFT OUTER JOIN authors a ON b.author_id = a.author_id
            GROUP BY 1,2,3,4
            HAVING COUNT(rating_id) >= 50)
            
SELECT author,
       AVG(avg_rating) true_avg_rating
FROM pt
GROUP BY 1
ORDER BY 2 DESC
LIMIT 1;
'''

pd.io.sql.read_sql(sql=text(query), con = engine)

,author,true_avg_rating
0,J.K. Rowling/Mary GrandPré,4.283844


*Вывод:* автор с самой высокой средней оценкой книг - J.K. Rowling/Mary GrandPré (4.28).

## Запрос №5

- Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [7]:
query = '''

WITH names AS (SELECT username
                FROM ratings
                GROUP BY 1
                HAVING COUNT(rating_id) > 50),
                
     counts AS (SELECT names.username,
                       COUNT(review_id) rv_count
                FROM names
                LEFT OUTER JOIN reviews rv ON names.username = rv.username
                GROUP BY 1)
                
SELECT FLOOR(AVG(rv_count)) 
FROM counts
''' 

pd.io.sql.read_sql(sql=text(query), con = engine)

,floor
0,24.0


*Вывод:* среднее количество обзоров от пользователей, которые поставили больше 50 оценок составляет 24.

## Итоговый вывод

В рамках проекта была проанализирована база данных, содержащая информацию о книгах, издательствах, авторах, а также пользовательских обзорах книг. Было проделано и установлено следующее:

- Запрос 1: После 1 января 2000 года вышло 819 книг.
- Запрос 2: для решения задачи была сформирована таблица, содержащая информацию об идентификаторах всех книг, их названиях, количестве обзоров и среднем значении рейтинга. Данные отсортированы по идентификаторам книг.
- Запрос 3: издательство, которое выпустило наибольшее число книг толще 50 страниц — Penguin Books (42 книги).
- Запрос 4: автор с самой высокой средней оценкой книг - J.K. Rowling/Mary GrandPré (4.28).
- Запрос 5: среднее количество обзоров от пользователей, которые поставили больше 50 оценок составляет 24.

Полученные данные помогут сформулировать ценностное предложение для нового продукта. Таким образом, цель проекта была достигнута.